In [1]:
# Dependencies
import json
import requests
import pandas as pd
import numpy as np
import os
from datetime import datetime
from api_keys import DarkSkyKey

In [2]:
# read taxi rides file
file = "Resources/Taxi_Trips_Random_Sample.csv"
df = pd.read_csv(file, header=None)
df.columns = ["Trip ID", "Taxi ID","Trip Start Timestamp", "Trip End Timestamp","Trip Seconds",
             "Trip Miles", "Pickup Census Tract", "Dropoff Census Tract", "Pickup Community Area", 
             "Dropoff Community Area", "Fare", "Tips", "Tolls", "Extras", "Trip Total", "Payment Type", 
             "Company", "Pickup Centroid Latitude", "Pickup Centroid Longitude", "Pickup Centroid Location", 
             "Dropoff Centroid Latitude", "Dropoff Centroid Longitude", "Dropoff Centroid Location","Community Areas"]
df

,Trip ID,Taxi ID,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Pickup Census Tract,Dropoff Census Tract,Pickup Community Area,Dropoff Community Area,...,Trip Total,Payment Type,Company,Pickup Centroid Latitude,Pickup Centroid Longitude,Pickup Centroid Location,Dropoff Centroid Latitude,Dropoff Centroid Longitude,Dropoff Centroid Location,Community Areas
0,bc6f08423e9f194cb63a0d5ec6c6a2dea001e914,07a82232d832859a7dcb1c40846bb3e21acb2c84755ef6...,11/16/2013 01:45:00 AM,11/16/2013 01:45:00 AM,480.0,0.15,1.703108e+10,1.703108e+10,8.0,8.0,...,8.05,Cash,NaN,41.907413,-87.640902,POINT (-87.6409015248 41.9074128162),41.892042,-87.631864,POINT (-87.6318639497 41.8920421365),37.0
1,b2e0ede2b0d907551beda4e8b4b83c143010f63d,74605d6aa0c8ba08190a5824f79ef27172c59671b00582...,09/28/2016 04:45:00 PM,09/28/2016 04:45:00 PM,420.0,0.00,1.703184e+10,1.703132e+10,32.0,32.0,...,8.75,Credit Card,Blue Ribbon Taxi Association Inc.,41.880994,-87.632746,POINT (-87.6327464887 41.8809944707),41.884987,-87.620993,POINT (-87.6209929134 41.8849871918),38.0
2,c2155aedc41a27673cd498d0fea754a18132d7f9,4243ab34318764eb5ca51054af503c610b50579594cef6...,12/10/2013 07:15:00 AM,12/10/2013 07:30:00 AM,720.0,1.80,1.703128e+10,1.703108e+10,28.0,8.0,...,8.44,Cash,NaN,41.879255,-87.642649,POINT (-87.642648998 41.8792550844),41.895033,-87.619711,POINT (-87.6197106717 41.8950334495),29.0
3,c1e860354f05150021e1bdeb7696314ca3e15608,65e345a16c02e0408409d6d71a7a5cb09449e5c84c16d0...,02/27/2014 09:00:00 AM,02/27/2014 09:45:00 AM,2700.0,0.00,1.703108e+10,1.703198e+10,8.0,76.0,...,50.70,Credit Card,Choice Taxi Association,41.902788,-87.626146,POINT (-87.6261455896 41.9027880476),41.979071,-87.903040,POINT (-87.9030396611 41.9790708201),37.0
4,bdff93a935333338e6f2f04f778ad7713a36c9ad,8ca267f2246008a052c14ce8c81001149d86916ab274d3...,10/31/2016 04:15:00 PM,10/31/2016 04:15:00 PM,660.0,3.00,NaN,NaN,32.0,35.0,...,13.50,Credit Card,NaN,41.878866,-87.625192,POINT (-87.6251921424 41.8788655841),41.835118,-87.618678,POINT (-87.6186777673 41.8351179863),38.0
5,be7313b87f53e20b46020458e7b019c58f80b223,87bb9f5f7f5f1b84620d05e616b62323c2aa945795ccda...,06/15/2016 09:30:00 AM,06/15/2016 09:45:00 AM,540.0,1.20,NaN,NaN,28.0,32.0,...,8.50,Credit Card,NaN,41.874005,-87.663518,POINT (-87.6635175498 41.874005383),41.878866,-87.625192,POINT (-87.6251921424 41.8788655841),29.0
6,b08cec8ff3328356d388071edbfcf3f0405edfc6,b0eee68bd58686cc729cf46fa27d775dac55168754f3c7...,09/16/2013 06:45:00 PM,09/16/2013 06:45:00 PM,600.0,1.50,1.703128e+10,1.703132e+10,28.0,32.0,...,7.85,Cash,NaN,41.879255,-87.642649,POINT (-87.642648998 41.8792550844),41.870607,-87.622173,POINT (-87.6221729369 41.8706073724),29.0
7,c189e7f30cc51e0eb24c7ddbf6d4c9adbdd9f2a7,c08461ef71e4e0f58fc9c2148cec4f79f67db2e569625b...,02/21/2014 11:45:00 PM,02/21/2014 11:45:00 PM,120.0,0.05,1.703107e+10,1.703107e+10,7.0,7.0,...,5.75,Cash,NaN,41.926811,-87.642605,POINT (-87.6426052473 41.9268111821),41.929047,-87.651311,POINT (-87.6513108767 41.9290469366),68.0
8,bc486809fdc9602698d026865cbfffafccbc0d11,29e9c1f8071508e15fe99c12fae2e0e25949a618458f23...,01/19/2013 02:15:00 AM,01/19/2013 02:30:00 AM,1080.0,0.00,1.703184e+10,1.703108e+10,8.0,8.0,...,13.65,Cash,Taxi Affiliation Services,41.904935,-87.649907,POINT (-87.6499072264 41.9049353016),41.892042,-87.631864,POINT (-87.6318639497 41.8920421365),37.0
9,b40db62cca32924d1b26ee28ea29b5dab6ccb242,a073b00b9ddc06e8def846986d51a0fef6f465fc87a061...,12/26/2014 10:45:00 PM,12/26/2014 11:00:00 PM,480.0,3.50,1.703106e+10,1.703103e+10,6.0,3.0,...,11.35,Cash,Taxi Affiliation Services,41.936310,-87.651563,POINT (-87.6515625922 41.9363101308),41.972929,-87.650290,POINT (-87.6502900743 41.9729293173),57.0


In [3]:
# convert timestamp format for api calls
newdatestart_list = []

for date in df['Trip Start Timestamp']:
    date_obj = datetime.strptime(date, '%m/%d/%Y %H:%M:%S %p')
    new_date = date_obj.strftime('%Y-%m-%dT%H:%M:%S')
    newdatestart_list.append(new_date)

df['Trip Start Timestamp'] = newdatestart_list
df.head()


,Trip ID,Taxi ID,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Pickup Census Tract,Dropoff Census Tract,Pickup Community Area,Dropoff Community Area,...,Trip Total,Payment Type,Company,Pickup Centroid Latitude,Pickup Centroid Longitude,Pickup Centroid Location,Dropoff Centroid Latitude,Dropoff Centroid Longitude,Dropoff Centroid Location,Community Areas
0,bc6f08423e9f194cb63a0d5ec6c6a2dea001e914,07a82232d832859a7dcb1c40846bb3e21acb2c84755ef6...,2013-11-16T01:45:00,11/16/2013 01:45:00 AM,480.0,0.15,1.703108e+10,1.703108e+10,8.0,8.0,...,8.05,Cash,NaN,41.907413,-87.640902,POINT (-87.6409015248 41.9074128162),41.892042,-87.631864,POINT (-87.6318639497 41.8920421365),37.0
1,b2e0ede2b0d907551beda4e8b4b83c143010f63d,74605d6aa0c8ba08190a5824f79ef27172c59671b00582...,2016-09-28T04:45:00,09/28/2016 04:45:00 PM,420.0,0.00,1.703184e+10,1.703132e+10,32.0,32.0,...,8.75,Credit Card,Blue Ribbon Taxi Association Inc.,41.880994,-87.632746,POINT (-87.6327464887 41.8809944707),41.884987,-87.620993,POINT (-87.6209929134 41.8849871918),38.0
2,c2155aedc41a27673cd498d0fea754a18132d7f9,4243ab34318764eb5ca51054af503c610b50579594cef6...,2013-12-10T07:15:00,12/10/2013 07:30:00 AM,720.0,1.80,1.703128e+10,1.703108e+10,28.0,8.0,...,8.44,Cash,NaN,41.879255,-87.642649,POINT (-87.642648998 41.8792550844),41.895033,-87.619711,POINT (-87.6197106717 41.8950334495),29.0
3,c1e860354f05150021e1bdeb7696314ca3e15608,65e345a16c02e0408409d6d71a7a5cb09449e5c84c16d0...,2014-02-27T09:00:00,02/27/2014 09:45:00 AM,2700.0,0.00,1.703108e+10,1.703198e+10,8.0,76.0,...,50.70,Credit Card,Choice Taxi Association,41.902788,-87.626146,POINT (-87.6261455896 41.9027880476),41.979071,-87.903040,POINT (-87.9030396611 41.9790708201),37.0
4,bdff93a935333338e6f2f04f778ad7713a36c9ad,8ca267f2246008a052c14ce8c81001149d86916ab274d3...,2016-10-31T04:15:00,10/31/2016 04:15:00 PM,660.0,3.00,NaN,NaN,32.0,35.0,...,13.50,Credit Card,NaN,41.878866,-87.625192,POINT (-87.6251921424 41.8788655841),41.835118,-87.618678,POINT (-87.6186777673 41.8351179863),38.0


In [4]:
# Save config information
url = "https://api.darksky.net/forecast/"
exception_count = 0

weather_df=pd.DataFrame()

# calculate begining of iteration from last line in csv file if existant
try:
    with open("Output/weather.csv", "r") as f:
        a = int(pd.read_csv("Output/weather.csv", header=None).tail(1)[0]+1)
except:
    a = 0

n = a
        
# Build query URL and DataFrame
for i in range(a, len(df)):
    #get info from taxi ride file
    lat = df['Pickup Centroid Latitude'][i]
    lng = df['Pickup Centroid Longitude'][i]
    time = df['Trip Start Timestamp'][i]

    #make API calls with this info and save to DataFrame
    try:
        query_url = url + DarkSkyKey + "/" + str(lat) +"," + str(lng) + "," + time 
        weather_response = requests.get(query_url)
        weather_json = weather_response.json()['currently'] 
        weather_df.loc[i,'Trip ID'] = df['Trip ID'][i]
        weather_df.loc[i,'time'] = time
        weather_df.loc[i,'summary'] = weather_json['summary']
        weather_df.loc[i,'precipIntensity'] = weather_json['precipIntensity']
        weather_df.loc[i,'precipProbability'] = weather_json['precipProbability']
        weather_df.loc[i,'temperature'] = weather_json['temperature']
        weather_df.loc[i,'apparentTemperature'] = weather_json['apparentTemperature']
        weather_df.loc[i,'windSpeed'] = weather_json['windSpeed']
        weather_df.loc[i,'precipProbability'] = weather_json['precipProbability']
        weather_df.loc[i,'icon'] = weather_json['icon']
    
    except:
        if i-n <= 1: exception_count += 1 # count consecutive failed calls
        else: exception_count = 0
            
        n = i
        
        if exception_count >= 5: 
            print(f"Request no {i} failed, 5 consecutive requests have failed, ending itteration")
            break
        else: 
            print(f"Request no {i} failed, skipping to next line")
weather_df

Request no 8105 failed, skipping to next line
Request no 8197 failed, skipping to next line
Request no 8240 failed, skipping to next line
Request no 8329 failed, skipping to next line
Request no 8354 failed, skipping to next line
Request no 8897 failed, skipping to next line
Request no 8898 failed, skipping to next line
Request no 8899 failed, skipping to next line
Request no 8900 failed, skipping to next line
Request no 8901 failed, skipping to next line
Request no 8902 failed, 5 consecutive requests have failed, ending itteration


,Trip ID,time,summary,precipIntensity,precipProbability,temperature,apparentTemperature,windSpeed,icon
7892,b31c8712f6dc2a658b6a3e585269bc7497f09281,2015-05-30T10:30:00,Drizzle,0.0067,0.75,63.49,63.75,1.17,rain
7893,c3ee0e69403cdd1d754b8f824edf7bf130fe90fb,2014-09-27T12:00:00,Mostly Cloudy,0.0000,0.00,74.79,74.79,1.99,partly-cloudy-day
7894,c3ebaa79c6322c1425b37527545978bdc26f4fd2,2014-10-27T01:15:00,Clear,0.0000,0.00,53.83,53.83,3.19,clear-night
7895,b0e973f7d33b1908bc825f80d8284c951d4e0e9d,2015-05-22T11:15:00,Clear,0.0000,0.00,61.14,61.14,2.81,clear-day
7896,bde583033880a2b7cfec19bd113207f60f02d261,2013-12-15T01:15:00,Overcast,0.0000,0.00,22.07,16.18,4.34,cloudy
7897,c0c3dc41cadaa0a5a0d5ca02808015a5335b423c,2013-05-28T05:30:00,Mostly Cloudy,0.0000,0.00,62.82,63.17,2.75,partly-cloudy-day
7898,c0d237aa7b684383042f66a85d801fed2b819fa4,2014-02-06T09:45:00,Clear,0.0000,0.00,3.34,-6.91,5.23,clear-day
7899,c0e4a08032b8c029b50c96af2ff16a9386e13f6e,2015-12-17T08:30:00,Overcast,0.0000,0.00,31.11,24.52,6.87,cloudy
7900,b9875565b650188e899cd9e16f76ce487f063a82,2016-11-10T01:00:00,Clear,0.0000,0.00,43.99,43.99,2.26,clear-night
7901,be1ffb16597c124986222456b8f0da5769e11e5f,2014-12-30T05:15:00,Clear,0.0000,0.00,14.39,7.40,4.24,clear-night


In [6]:
# Save results in csv output file (append if file already exists)
try: os.mkdir('Output')
except: pass

try:
    with open("Output/weather.csv", "a") as f:
        weather_df.to_csv(f, header=False)
except:
    weather_df.to_csv("Output/weather.csv")